In [1]:
import os
from glob import glob
import re 
import math
from pathlib import Path 
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 


import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import mne
mne.set_log_level(verbose=False)
import wandb

from Datasets import *

from utils_folder.callbacks import *
from utils_folder.training_pth import *
from utils_folder.plotting import *
from utils_folder.utils import get_labels_and_preds

In [2]:
def get_order(file_path):
    """
    Used to order the results from glob, so that the patients are
    properly concatenated.
    """
    match = file_pattern.match(Path(file_path).name)
    if not match:
        return math.inf
    return int(match.groups()[0])

In [ ]:
## Define the regex to sort the files and obtain the paths list
file_pattern = re.compile(r'.*?(\d+).*?')
sorted_files = sorted(glob("/media/usbdisk/data/ProyectoPSG/data/*.edf"), key=get_order)
### Remove the 10th patient
sorted_files_no_10 = [a for a in sorted_files if re.findall(r'\d+', a)[0]!='10']

## Load the data
datasets = [EDFData_PTH(path_glob, channels=config.channels, binary=config.binary) for path_glob in sorted_files_no_10]